# Giesekus Stress Relaxation: Faster-than-Maxwell Decay

## Learning Objectives

1. Fit the Giesekus model to real polymer stress relaxation data
2. Understand how the τ·τ term accelerates stress decay
3. Compare Giesekus relaxation to single-exponential Maxwell
4. Observe the effect of α on relaxation shape
5. Extract relaxation modulus G(t) from the data

## Prerequisites

- Basic RheoJAX usage (basic/01-maxwell-fitting.ipynb)
- Bayesian inference fundamentals (bayesian/01-bayesian-basics.ipynb)

## Runtime

- Fast demo (NUM_CHAINS=1, NUM_SAMPLES=500): ~3-5 minutes
- Full run (NUM_CHAINS=4, NUM_SAMPLES=2000): ~15-20 minutes

## 1. Setup

In [ ]:
# Google Colab setup
import sys

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax
    import os
    os.environ["JAX_ENABLE_X64"] = "true"
    print("RheoJAX installed successfully.")

In [ ]:
# Imports
%matplotlib inline
import json
import os
import time
import warnings

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.giesekus import GiesekusSingleMode

jax, jnp = safe_import_jax()
verify_float64()

warnings.filterwarnings("ignore", category=FutureWarning)
print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")

## 2. Theory: Stress Relaxation in Giesekus

### The Experiment

In a stress relaxation test, we apply a constant shear rate γ̇ until steady state, then suddenly stop (γ̇ → 0) and measure the stress decay σ(t).

### Maxwell Relaxation

For a Maxwell fluid:
$$
\sigma(t) = \sigma_0 \exp(-t/\lambda)
$$

where σ₀ is the stress at t = 0 (when shear stops).

### Giesekus Relaxation

For the Giesekus model, the relaxation is **faster than exponential** due to the quadratic τ·τ term. The stress decays as:

$$
\frac{d\boldsymbol{\tau}}{dt} = -\frac{1}{\lambda}\left(\boldsymbol{\tau} + \frac{\alpha}{\eta_p}\boldsymbol{\tau}\cdot\boldsymbol{\tau}\right)
$$

The τ·τ term provides **additional dissipation** that accelerates decay.

### Key Features

| Model | Relaxation Shape | Short Time | Long Time |
|-------|-----------------|------------|------------|
| Maxwell | Exponential | σ ~ σ₀ | σ ~ exp(-t/λ) |
| Giesekus (α > 0) | Faster-than-exponential | Similar to Maxwell | Faster decay |
| Giesekus (α = 0) | Maxwell | (UCM limit) | |

### Relaxation Modulus

The relaxation modulus is defined as:
$$
G(t) = \frac{\sigma(t)}{\gamma_0}
$$

where γ₀ is a small applied strain.

## 3. Load Data

We use polystyrene stress relaxation data at T = 145°C.

In [ ]:
# Load polymer relaxation data
data_path = os.path.join("..", "data", "relaxation", "polymers", "stressrelaxation_ps145_data.csv")

# Tab-separated with header
raw = np.loadtxt(data_path, delimiter="\t", skiprows=1)
time_data = raw[:, 0]  # Time (s)
G_data = raw[:, 1]  # Relaxation modulus (Pa)

print(f"Loaded {len(time_data)} data points")
print(f"Time range: [{time_data.min():.3f}, {time_data.max():.1f}] s")
print(f"G(t) range: [{G_data.min():.2f}, {G_data.max():.2f}] Pa")

In [ ]:
# Plot raw data
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Log-log scale
ax1.loglog(time_data, G_data, "ko", markersize=5)
ax1.set_xlabel("Time [s]")
ax1.set_ylabel("Relaxation modulus G(t) [Pa]")
ax1.set_title("Polystyrene Stress Relaxation (T=145°C)")
ax1.grid(True, alpha=0.3, which="both")

# Semi-log (to check for exponential)
ax2.semilogy(time_data, G_data, "ko", markersize=5)
ax2.set_xlabel("Time [s]")
ax2.set_ylabel("Relaxation modulus G(t) [Pa]")
ax2.set_title("Semi-Log Plot (straight line = exponential)")
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

## 4. NLSQ Fitting

In [ ]:
# Create and fit Giesekus model
model = GiesekusSingleMode()

# For relaxation, we need to specify the pre-shear rate
gamma_dot_preshear = 1.0  # 1/s (assumed)

t0 = time.time()
model.fit(time_data, G_data, test_mode="relaxation", gamma_dot=gamma_dot_preshear, method='scipy')
t_nlsq = time.time() - t0

print(f"NLSQ fit time: {t_nlsq:.2f} s")
print(f"\nFitted parameters:")
param_names = ["eta_p", "lambda_1", "alpha", "eta_s"]
for name in param_names:
    val = model.parameters.get_value(name)
    print(f"  {name:10s} = {val:.4g}")

# Derived quantities
eta_p = model.parameters.get_value("eta_p")
eta_s = model.parameters.get_value("eta_s")
lambda_1 = model.parameters.get_value("lambda_1")
alpha = model.parameters.get_value("alpha")

G_0 = eta_p / lambda_1

print(f"\nDerived quantities:")
print(f"  G₀ = η_p/λ = {G_0:.2f} Pa")
print(f"  Relaxation time λ = {lambda_1:.3f} s")

In [ ]:
# Plot fit with data
time_fine = np.logspace(
    np.log10(time_data.min()),
    np.log10(time_data.max()),
    200,
)
G_pred = model.predict(time_fine, test_mode="relaxation", gamma_dot=gamma_dot_preshear)

# Also compute Maxwell for comparison
G_maxwell = G_0 * np.exp(-time_fine / lambda_1)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Log-log
ax1.loglog(time_data, G_data, "ko", markersize=5, label="Data")
ax1.loglog(time_fine, G_pred, "-", lw=2, color="C0", label="Giesekus fit")
ax1.loglog(time_fine, G_maxwell, "--", lw=2, color="C1", alpha=0.7, label="Maxwell (same λ)")
ax1.set_xlabel("Time [s]")
ax1.set_ylabel("G(t) [Pa]")
ax1.set_title("Relaxation Modulus (Log-Log)")
ax1.legend()
ax1.grid(True, alpha=0.3, which="both")

# Semi-log
ax2.semilogy(time_data, G_data, "ko", markersize=5, label="Data")
ax2.semilogy(time_fine, G_pred, "-", lw=2, color="C0", label="Giesekus fit")
ax2.semilogy(time_fine, G_maxwell, "--", lw=2, color="C1", alpha=0.7, label="Maxwell")
ax2.set_xlabel("Time [s]")
ax2.set_ylabel("G(t) [Pa]")
ax2.set_title("Relaxation Modulus (Semi-Log)")
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

## 5. Effect of α on Relaxation

In [ ]:
# Compare different α values
alpha_values = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
colors = plt.cm.viridis(np.linspace(0.1, 0.9, len(alpha_values)))

# Use fitted parameters but vary α
time_sim = np.linspace(0.01, 5 * lambda_1, 200)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

for i, alpha_i in enumerate(alpha_values):
    model_i = GiesekusSingleMode()
    model_i.parameters.set_value("eta_p", eta_p)
    model_i.parameters.set_value("lambda_1", lambda_1)
    model_i.parameters.set_value("alpha", alpha_i)
    model_i.parameters.set_value("eta_s", eta_s)
    
    G_i = np.array(model_i.simulate_relaxation(time_sim, gamma_dot_preshear=gamma_dot_preshear))
    G_i_norm = G_i / G_i[0]  # Normalize
    
    label = "UCM" if alpha_i == 0 else f"α = {alpha_i}"
    
    ax1.semilogy(time_sim / lambda_1, G_i_norm, "-", lw=2, color=colors[i], label=label)
    ax2.plot(time_sim / lambda_1, G_i_norm, "-", lw=2, color=colors[i], label=label)

ax1.set_xlabel("Dimensionless time t/λ")
ax1.set_ylabel("Normalized G(t)/G(0)")
ax1.set_title("Effect of α on Relaxation (Semi-Log)")
ax1.legend(fontsize=9)
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, 5)

ax2.set_xlabel("Dimensionless time t/λ")
ax2.set_ylabel("Normalized G(t)/G(0)")
ax2.set_title("Effect of α on Relaxation (Linear)")
ax2.legend(fontsize=9)
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0, 5)
ax2.set_ylim(0, 1.1)

plt.tight_layout()
display(fig)
plt.close(fig)

print("Key observation: Higher α → faster relaxation")
print("The τ·τ term provides additional dissipation.")

## 6. Bayesian Inference

In [ ]:
# Bayesian inference with NLSQ warm-start
initial_values = {
    name: model.parameters.get_value(name)
    for name in param_names
}
print("Warm-start values:")
for k, v in initial_values.items():
    print(f"  {k}: {v:.4g}")

# Fast demo config
NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1
# NUM_WARMUP = 1000; NUM_SAMPLES = 2000; NUM_CHAINS = 4  # production

t0 = time.time()
result = model.fit_bayesian(
    time_data,
    G_data,
    test_mode="relaxation",
    gamma_dot=gamma_dot_preshear,
    num_warmup=NUM_WARMUP,
    num_samples=NUM_SAMPLES,
    num_chains=NUM_CHAINS,
    initial_values=initial_values,
    seed=42,
)
t_bayes = time.time() - t0
print(f"\nBayesian inference time: {t_bayes:.1f} s")

In [ ]:
# Convergence diagnostics
diag = result.diagnostics

print("Convergence Diagnostics")
print("=" * 55)
print(f"{'Parameter':>12s}  {'R-hat':>8s}  {'ESS':>8s}  {'Status':>8s}")
print("-" * 55)

for p in param_names:
    r_hat = diag.get("r_hat", {}).get(p, float("nan"))
    ess = diag.get("ess", {}).get(p, float("nan"))
    status = "PASS" if (r_hat < 1.05 and ess > 100) else "CHECK"
    print(f"{p:>12s}  {r_hat:8.4f}  {ess:8.0f}  {status:>8s}")

n_div = diag.get("divergences", diag.get("num_divergences", 0))
print(f"\nDivergences: {n_div}")

In [ ]:
# ArviZ trace plots
idata = result.to_inference_data()

axes = az.plot_trace(idata, var_names=param_names, figsize=(12, 8))
fig = axes.ravel()[0].figure
fig.suptitle("Trace Plots", fontsize=14, y=1.02)
plt.tight_layout()
display(fig)
plt.close(fig)

In [ ]:
# Forest plot
axes = az.plot_forest(
    idata,
    var_names=param_names,
    combined=True,
    hdi_prob=0.95,
    figsize=(10, 5),
)
fig = axes.ravel()[0].figure
plt.tight_layout()
display(fig)
plt.close(fig)

## 7. Parameter Summary

In [ ]:
# NLSQ vs Bayesian comparison
posterior = result.posterior_samples

print("Parameter Comparison: NLSQ vs Bayesian")
print("=" * 70)
print(f"{'Param':>12s}  {'NLSQ':>12s}  {'Bayes median':>14s}  {'95% CI':>26s}")
print("-" * 70)

for name in param_names:
    nlsq_val = initial_values[name]
    bayes_samples = posterior[name]
    median = float(np.median(bayes_samples))
    lo = float(np.percentile(bayes_samples, 2.5))
    hi = float(np.percentile(bayes_samples, 97.5))
    print(f"{name:>12s}  {nlsq_val:12.4g}  {median:14.4g}  [{lo:.4g}, {hi:.4g}]")

## 8. Save Results

In [ ]:
# Save results
output_dir = os.path.join("..", "outputs", "giesekus", "relaxation")
os.makedirs(output_dir, exist_ok=True)

# Save NLSQ point estimates
nlsq_params = initial_values.copy()
with open(os.path.join(output_dir, "nlsq_params_relaxation.json"), "w") as f:
    json.dump(nlsq_params, f, indent=2)

# Save posterior samples
posterior_dict = {k: np.array(v).tolist() for k, v in posterior.items()}
with open(os.path.join(output_dir, "posterior_relaxation.json"), "w") as f:
    json.dump(posterior_dict, f)

print(f"Results saved to {output_dir}/")

## Key Takeaways

1. **Giesekus relaxation is faster than Maxwell** due to the quadratic τ·τ term. The additional dissipation accelerates stress decay.

2. **α controls the deviation from exponential**. Higher α → faster relaxation. At α = 0 (UCM), Giesekus reduces to single-exponential Maxwell.

3. **Semi-log plot diagnostic**: A straight line indicates exponential relaxation (Maxwell). Curvature indicates non-exponential behavior (Giesekus with α > 0).

4. **Multi-mode extension**: Real polymers often show broad relaxation spectra requiring multiple modes.

5. **Relaxation modulus** G(t) is related to complex modulus G*(ω) via Fourier transform.

### Experimental Notes

- Stress relaxation tests require sudden strain application (step strain)
- Instrument inertia can affect short-time response
- Sample loading and pre-shear history matter

### Next Steps

- **NB 07**: LAOS (nonlinear oscillatory response)